In [171]:
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget
import zhinst.core
import pymongo

In [172]:
device_id = "dev4934" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection or when MFLI/MFIA is connected to a remote Data Server.
# interface = "USB" # For all instruments connected to the host computer via USB except MFLI/MFIA.
#interface = "PCIe" # For MFLI/MFIA devices in case the Data Server runs on the device.

# server_host = "localhost"
server_host = '127.0.0.1'
# server_host = "192.168.77.26"
server_port = 8004

#server_port = 8005 # Default port for HF2LI.
api_level = 6 # Maximum API level supported for all instruments except HF2LI.
#api_level = 1 # Maximum API level supported for HF2LI.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

In [173]:
demod_path = f"/{device_id}/demods/0/"
sample_path = demod_path+'sample'

sample =daq.getSample(sample_path)
display(sample)

{'timestamp': array([193766081381967], dtype=uint64),
 'x': array([-3.72383918e-12]),
 'y': array([-1.25061777e-12]),
 'frequency': array([733.99999998]),
 'phase': array([5.2999995]),
 'dio': array([0], dtype=uint32),
 'trigger': array([0], dtype=uint32),
 'auxin0': array([0.00230617]),
 'auxin1': array([-0.00131363])}

In [174]:
daq.subscribe(sample_path)
# daq.poll()
poll_res =daq.poll(recording_time_s=0.1, timeout_ms=500)
display(poll_res)
poll_sample=poll_res['dev4934']['demods']['0']['sample']

{'dev4934': {'demods': {'0': {'sample': {'timestamp': array([193766087367247, 193766087403087, 193766087438927, 193766087474767,
            193766087510607, 193766087546447, 193766087582287, 193766087618127,
            193766087653967, 193766087689807, 193766087725647, 193766087761487,
            193766087797327, 193766087833167, 193766087869007, 193766087904847,
            193766087940687, 193766087976527, 193766088012367, 193766088048207,
            193766088084047, 193766088119887, 193766088155727, 193766088191567,
            193766088227407, 193766088263247, 193766088299087, 193766088334927,
            193766088370767, 193766088406607, 193766088442447, 193766088478287,
            193766088514127, 193766088549967, 193766088585807, 193766088621647,
            193766088657487, 193766088693327, 193766088729167, 193766088765007,
            193766088800847, 193766088836687, 193766088872527, 193766088908367,
            193766088944207, 193766088980047, 193766089015887, 19376608

In [175]:
from pymongo import MongoClient
import json
mongo_client = MongoClient(tz_aware=True)
db = mongo_client['test']
coll = 'zurich'
if coll not in db.list_collection_names():
    db.create_collection(coll)

In [176]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(CustomEncoder, self).default(obj)
        
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [177]:
poll_json =json.dumps(poll_sample, cls = NumpyEncoder)
poll_dict = json.loads(poll_json)
poll_dict

{'timestamp': [193766087367247,
  193766087403087,
  193766087438927,
  193766087474767,
  193766087510607,
  193766087546447,
  193766087582287,
  193766087618127,
  193766087653967,
  193766087689807,
  193766087725647,
  193766087761487,
  193766087797327,
  193766087833167,
  193766087869007,
  193766087904847,
  193766087940687,
  193766087976527,
  193766088012367,
  193766088048207,
  193766088084047,
  193766088119887,
  193766088155727,
  193766088191567,
  193766088227407,
  193766088263247,
  193766088299087,
  193766088334927,
  193766088370767,
  193766088406607,
  193766088442447,
  193766088478287,
  193766088514127,
  193766088549967,
  193766088585807,
  193766088621647,
  193766088657487,
  193766088693327,
  193766088729167,
  193766088765007,
  193766088800847,
  193766088836687,
  193766088872527,
  193766088908367,
  193766088944207,
  193766088980047,
  193766089015887,
  193766089051727,
  193766089087567,
  193766089123407,
  193766089159247,
  193766089195087,

In [180]:
mydict = { "name": "Peter", "address": "Lowstreet 27" }
db[coll].insert_one(poll_dict)

InsertOneResult(ObjectId('65750cd1fd5fbcf66760ed97'), acknowledged=True)

In [ ]:
daq.setDouble('dev4934/oscs/0/freq', 734)



In [ ]:
'/dev4934/demods/0/sample.R'

'/dev4934/demods/0/sample.R'

In [ ]:
daq.getSample(demod_path+'sample')


{'timestamp': array([227374896695982], dtype=uint64),
 'x': array([1.72026712e-09]),
 'y': array([-1.18313107e-09]),
 'frequency': array([732.9999999]),
 'phase': array([2.07413377]),
 'dio': array([0], dtype=uint32),
 'trigger': array([768], dtype=uint32),
 'auxin0': array([0.00164727]),
 'auxin1': array([-0.00098523])}

In [ ]:
display(daq.listNodes(demod_path))
display(daq.getSample(sample_path))

['/dev4934/demods/0/adcselect',
 '/dev4934/demods/0/order',
 '/dev4934/demods/0/rate',
 '/dev4934/demods/0/oscselect',
 '/dev4934/demods/0/harmonic',
 '/dev4934/demods/0/phaseshift',
 '/dev4934/demods/0/sinc',
 '/dev4934/demods/0/bypass',
 '/dev4934/demods/0/timeconstant',
 '/dev4934/demods/0/enable',
 '/dev4934/demods/0/freq',
 '/dev4934/demods/0/trigger',
 '/dev4934/demods/0/phaseadjust',
 '/dev4934/demods/0/sample']

{'timestamp': array([227375330610862], dtype=uint64),
 'x': array([-6.90321325e-10]),
 'y': array([-1.45805114e-09]),
 'frequency': array([732.9999999]),
 'phase': array([2.03300391]),
 'dio': array([0], dtype=uint32),
 'trigger': array([256], dtype=uint32),
 'auxin0': array([0.00098836]),
 'auxin1': array([-0.00065682])}

In [ ]:
daq.getSample('/dev4934/demods/0/sample')

{'timestamp': array([227375541708462], dtype=uint64),
 'x': array([-5.79671162e-10]),
 'y': array([2.92274571e-09]),
 'frequency': array([732.9999999]),
 'phase': array([1.46130845]),
 'dio': array([0], dtype=uint32),
 'trigger': array([768], dtype=uint32),
 'auxin0': array([0.]),
 'auxin1': array([-0.00164204])}

In [ ]:
# daq.setDouble(f'/{device_id}/sigouts/0/range',12E-3)



In [ ]:
'{:}'.format(3)

'3'

In [ ]:
import datetime as dt
dt.datetime.utcnow().timestamp()

1686521581.199947

# zi_MFLI class

In [ ]:
from importlib import reload
import zurich_MFLI as zur_driver
reload(zur_driver)

daq_obj = zur_driver.zi_MFLI(device_id, interface, server_host, server_port, api_level)

In [ ]:
display(daq_obj.get_XY())
display(daq_obj.get_freq())

[array([7.05867046e-06]), array([2.38378174e-07])]

array([32764.49999994])

In [ ]:
daq_obj.set_freq(32764.51)
daq_obj.get_freq()

array([32764.51000005])